In [2]:
import pandas as pd

df = pd.read_csv('top_movies_cleaned.csv') # read csv file

Create a DataFrame for the table in SQL with the cast id, name and gender:

In [3]:
df_act = pd.read_csv('API_cast_gender.csv') # read csv file
df_dir = pd.read_csv('API_dir_gender.csv') # read csv file

In [4]:
df_act['person_gender'] = df_act['person_gender'].replace({'Hombre': 'M', 'Mujer': 'F'})

In [5]:
df_act.rename(columns={'person_id': 'actor_id', 'person_name': 'actor_name', 'person_gender': 'actor_gender'}, inplace=True)

In [6]:
df_act.sample(10)

,actor_id,actor_name,actor_gender
2421,2422,Nina Dobrev,F
1556,1557,John Candy,M
503,504,Channing Tatum,M
2397,2398,Nicholas Clay,M
1311,1312,Jack Davenport,M
260,261,Awkwafina,F
1486,1487,Jesse Bradford,M
1349,1350,Jake Gyllenhaal,M
356,357,Blanche Baker,F
317,318,Bert Lahr,M


In [7]:
df_act.to_csv('actors.csv', index=False)

Create a DataFrame for the table in SQL with the director id, name and gender:

In [8]:
df_dir['person_gender'] = df_dir['person_gender'].replace({'Hombre': 'M', 'Mujer': 'F'})

In [9]:
df_dir.rename(columns={'person_id': 'director_id', 'person_name': 'director_name', 'person_gender': 'director_gender'}, inplace=True)

In [10]:
df_dir.sample(5)

,director_id,director_name,director_gender
831,832,Sydney Pollack,M
54,55,Annabel Jankel,F
587,588,Michael Matthews,M
381,382,Jenny Gage,F
353,354,James Wan,M


In [11]:
df_dir.to_csv('directors.csv', index=False)

Create he dataframe to the genres table, with the id and genre name:

In [12]:
import ast
df['genre'] = df['genre'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [13]:
all_genres = [genre for sublist in df['genre'] for genre in sublist]

unique_genres = pd.unique(all_genres)

genres_df = pd.DataFrame(unique_genres, columns=['genre_name'])

C:\Users\almuo\AppData\Local\Temp\ipykernel_11436\3372808026.py:3: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_genres = pd.unique(all_genres)


In [14]:
genres_df['genre_id'] = genres_df.index + 1

In [33]:
genres_df.head(10)

,genre_name,genre_id
0,Drama,1
1,Mystery,2
2,Thriller,3
3,Adventure,4
4,Comedy,5
5,Family,6
6,Romance,7
7,Crime,8
8,History,9
9,Action,10


In [16]:
genres_df.to_csv('genres.csv', index=False)

Create the dataframe for 'movies'table with all the basic information with 'movie_id', 'title', 'rating', 'meta_score', 'year', 'revenue', 'director_id'

In [17]:
# Make a copy of the dataframe to avoid modifying the original data
movies_df = df[['title', 'rating', 'meta_score', 'year', 'revenue', 'director']].copy()

In [18]:
#Add the movie_id column
movies_df['movie_id'] = movies_df.index + 1

In [19]:
# Translate the director's name to director_id
movies_df = movies_df.merge(df_dir[['director_id', 'director_name']], left_on='director', right_on='director_name', how='left')

In [20]:
#Select only the columns we need
movies_df = movies_df[['movie_id', 'title', 'rating', 'meta_score', 'year', 'revenue', 'director_id']]

In [21]:
movies_df.sample(10)

,movie_id,title,rating,meta_score,year,revenue,director_id
1509,1510,heaven help us,6.9,64.0,1985,0,578
574,575,the bishops wife,7.6,73.0,1947,0,323
915,916,remember me,7.1,40.0,2010,56032889,32
482,483,once upon a time in america,8.3,75.0,1984,5500000,788
326,327,nobody,7.4,64.0,2021,61779301,331
471,472,american history x,8.5,62.0,1998,23875127,881
1100,1101,dr. strangelove or how i learned to stop worry...,8.4,97.0,1964,9523464,810
231,232,the notebook,7.8,53.0,2004,115603229,627
1173,1174,the great outdoors,6.6,24.0,1988,43455230,327
536,537,batman returns,7.1,68.0,1992,280000000,856


In [22]:
movies_df.to_csv('movies.csv', index=False)

Create dataframe for cast table with 'movie_id' and 'person_id':

In [23]:
df['cast'] = df['cast'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [24]:
cast_df = df.merge(movies_df[['movie_id', 'title']], how='left', left_on='title', right_on='title')

In [25]:
cast_df = cast_df[['movie_id', 'cast']].explode('cast').rename(columns={'cast': 'actor_name'})

In [26]:
cast_df = cast_df.merge(df_act, how='left', left_on='actor_name', right_on='actor_name')

In [27]:
cast_df = cast_df[['movie_id', 'actor_id']]

In [28]:
cast_df.head(10)

,movie_id,actor_id
0,1,1707
1,1,2081
2,1,1004
3,1,2355
4,2,3120
5,2,1180
6,2,2354
7,2,2520
8,3,972
9,3,2135


In [29]:
cast_df.to_csv('cast.csv', index=False)

Create a dataframe wich relates movies_id and genres_id

In [30]:
df['genre'] = df['genre'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

movie_genres_df = df.merge(movies_df[['movie_id', 'title']], how='left', left_on='title', right_on='title')

movie_genres_df = movie_genres_df[['movie_id', 'genre']].explode('genre').rename(columns={'genre': 'genre_name'})

movie_genres_df = movie_genres_df.merge(genres_df, how='left', left_on='genre_name', right_on='genre_name')

movie_genres_df = movie_genres_df[['movie_id', 'genre_id']]

In [31]:
movie_genres_df.head(10)

,movie_id,genre_id
0,1,1
1,1,2
2,1,3
3,2,4
4,2,5
5,2,6
6,3,5
7,3,1
8,3,7
9,4,8


In [32]:
movie_genres_df.to_csv('movie_genres.csv', index=False)